In [1]:
# %%
from dataclasses import dataclass
from datetime import datetime, timezone, timedelta
from pathlib import Path

import numpy as np
import pandas as pd

from hopeit.dataobjects.payload import Payload
from hopeit.dataframes import dataframe

DATA_PATH = Path("forecast_example_data")

# %%
@dataframe
@dataclass
class OutboundForecast:
    warehouse_id: int
    date: datetime
    ob_units: float


@dataframe
@dataclass
class Dwell:
    days_since_open: int
    dwell_days: int
    ratio: float

@dataframe
@dataclass
class Warehouse:
    warehouse_id: int
    start_date: datetime


In [2]:

warehouses = Warehouse.from_df(
    pd.read_csv(DATA_PATH / "warehouses.csv")
)
ob_forecast = OutboundForecast.from_df(
    pd.read_csv(DATA_PATH / "outbound_forecast.csv" )
)
dwell_config = Dwell.from_df(
    pd.read_csv(DATA_PATH / "dwell_std.csv")
)

pre coerce warehouse_id     int64
start_date      object
dtype: object
post coerce warehouse_id             int64
start_date      datetime64[ns]
dtype: object
pre coerce warehouse_id      int64
date             object
ob_units        float64
dtype: object
post coerce warehouse_id             int64
date            datetime64[ns]
ob_units               float64
dtype: object
pre coerce days_since_open      int64
dwell_days           int64
ratio              float64
dtype: object
post coerce days_since_open      int64
dwell_days           int64
ratio              float64
dtype: object


In [3]:
@dataframe
@dataclass
class ForecastDwell:
    warehouse_id: int
    date: datetime
    ob_units: float
    start_date: datetime
    days_since_open: int
    dwell_days: int
    ratio: float

join_df =ob_forecast.df.merge(warehouses.df, on="warehouse_id")

forecast_dwell = ForecastDwell.from_df(
    join_df,
    days_since_open=(join_df.date - join_df.start_date) / np.timedelta64(1, 'D'),
    dwell_days=0,  # Default value
    ratio=0.0  # Default value
)


pre coerce warehouse_id                int64
date               datetime64[ns]
ob_units                  float64
start_date         datetime64[ns]
days_since_open           float64
dwell_days                  int64
ratio                     float64
dtype: object
post coerce warehouse_id                int64
date               datetime64[ns]
ob_units                  float64
start_date         datetime64[ns]
days_since_open             int64
dwell_days                  int64
ratio                     float64
dtype: object


In [53]:
df = dwell_config.df.merge(
    forecast_dwell.df, how="outer", on="days_since_open", suffixes=("", "_default")
).sort_values(["days_since_open"])

df["dwell_days"] = df["dwell_days"].ffill()
df["ratio"] = df["ratio"].ffill()

forecast_dwell = ForecastDwell.from_df(
    df.dropna()
)

forecast_dwell.df

pre coerce warehouse_id              float64
date               datetime64[ns]
ob_units                  float64
start_date         datetime64[ns]
days_since_open             int64
dwell_days                float64
ratio                     float64
dtype: object
post coerce warehouse_id                int64
date               datetime64[ns]
ob_units                  float64
start_date         datetime64[ns]
days_since_open             int64
dwell_days                  int64
ratio                     float64
dtype: object


/Users/hopeit/dev/leosmerling/hopeit.engine/plugins/data/dataframes/src/hopeit/dataframes/__init__.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.__df[field.name] = self.DATATYPE_MAPPING[field.type](
/Users/hopeit/dev/leosmerling/hopeit.engine/plugins/data/dataframes/src/hopeit/dataframes/__init__.py:127: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.__df[field.name] = self.DATATYPE_MAPPING[field.type](
/Users/hopeit/dev/leosmerling/hopeit.engine/plugins/data/dataframes/src/hopeit/dataframe

,warehouse_id,date,ob_units,start_date,days_since_open,dwell_days,ratio
0,1,2023-01-01,1000.0,2023-01-01,0,7,0.1
4,1,2023-02-01,1500.0,2023-01-01,31,5,0.3
5,1,2023-03-01,2000.0,2023-01-01,59,5,0.3
3,1,2023-04-01,1800.0,2023-01-01,90,2,1.0
6,1,2023-05-01,1700.0,2023-01-01,120,2,1.0
7,1,2023-06-01,1600.0,2023-01-01,151,2,1.0


In [54]:
df = forecast_dwell.df
print(df.dtypes)

df["inbound_end_date"] = df["date"] - df["dwell_days"] * pd.DateOffset(days=1)
df["inbound_start_date"] = df["inbound_end_date"] - (1 / df["ratio"] - 1).round(0).astype(int) * pd.DateOffset(days=1)
df["inbound_date"] = df.apply(lambda x: pd.date_range(x["inbound_start_date"], x["inbound_end_date"]), axis=1)
df["ib_units"] = df["ob_units"] * df["ratio"]


df.explode("inbound_date")

warehouse_id                int64
date               datetime64[ns]
ob_units                  float64
start_date         datetime64[ns]
days_since_open             int64
dwell_days                  int64
ratio                     float64
dtype: object


/var/folders/_v/0wf_45f91qn7lm7tfrygw6fr0000gn/T/ipykernel_2589/3064474874.py:4: PerformanceWarning: Adding/subtracting object-dtype array to DatetimeArray not vectorized.
  df["inbound_end_date"] = df["date"] - df["dwell_days"] * pd.DateOffset(days=1)


,warehouse_id,date,ob_units,start_date,days_since_open,dwell_days,ratio,inbound_end_date,inbound_start_date,inbound_date,ib_units
0,1,2023-01-01,1000.0,2023-01-01,0,7,0.1,2022-12-25 00:00:00,2022-12-16 00:00:00,2022-12-16,100.0
0,1,2023-01-01,1000.0,2023-01-01,0,7,0.1,2022-12-25 00:00:00,2022-12-16 00:00:00,2022-12-17,100.0
0,1,2023-01-01,1000.0,2023-01-01,0,7,0.1,2022-12-25 00:00:00,2022-12-16 00:00:00,2022-12-18,100.0
0,1,2023-01-01,1000.0,2023-01-01,0,7,0.1,2022-12-25 00:00:00,2022-12-16 00:00:00,2022-12-19,100.0
0,1,2023-01-01,1000.0,2023-01-01,0,7,0.1,2022-12-25 00:00:00,2022-12-16 00:00:00,2022-12-20,100.0
0,1,2023-01-01,1000.0,2023-01-01,0,7,0.1,2022-12-25 00:00:00,2022-12-16 00:00:00,2022-12-21,100.0
0,1,2023-01-01,1000.0,2023-01-01,0,7,0.1,2022-12-25 00:00:00,2022-12-16 00:00:00,2022-12-22,100.0
0,1,2023-01-01,1000.0,2023-01-01,0,7,0.1,2022-12-25 00:00:00,2022-12-16 00:00:00,2022-12-23,100.0
0,1,2023-01-01,1000.0,2023-01-01,0,7,0.1,2022-12-25 00:00:00,2022-12-16 00:00:00,2022-12-24,100.0
0,1,2023-01-01,1000.0,2023-01-01,0,7,0.1,2022-12-25 00:00:00,2022-12-16 00:00:00,2022-12-25,100.0
